In [28]:
!pip install pandas regex numpy google-cloud-storage

In [29]:
import pandas as pd
import re
import numpy as np

# Colab auth
from google.colab import auth
from google.cloud import storage

In [30]:
#auth user in GCP
auth.authenticate_user()

In [31]:
!gcloud config set project 'steel-climber-398320'

Updated property [core/project].


In [32]:
# initialize the GCS client
client = storage.Client()

In [33]:
def download_blob(bucket_name, source_blob_name, destination_file_name):
    """downloads from the bucket."""
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """uploads to the bucket."""
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)

In [34]:
# Download data from GCS
download_blob('scraped_data_raw', 'nhs/training_data.json', './nhs_data.json')
download_blob('scraped_data_raw', 'mayo_clinic/mayo_clinic_data.json', './mayo_data.json')

# Process data
db1 = pd.read_json('nhs_data.json')
db2 = pd.read_json('mayo_data.json')

db1['prompt'] = db1['prompt'].str.replace('->', '', regex=False)
db2['prompt'] = db2['prompt'].str.replace('->', '', regex=False)

db1['completion'] = db1['completion'].str.replace('\n', '', regex=False)
db2['completion'] = db2['completion'].str.replace('\\n', '', regex=False)

db1.columns = ['symptoms', 'diagnosis']
db2.columns = ['symptoms', 'diagnosis']

In [35]:
# Save to local storage
db1.to_json('./nhs_data_clean.json', orient='records', lines=True)
db2.to_json('./mayo_clinic_data_clean.json', orient='records', lines=True)

# Upload processed data back to GCS bucket
upload_blob('scraped_data_clean', './nhs_data_clean.json', 'nhs/nhs_data_clean.json')
upload_blob('scraped_data_clean', './mayo_clinic_data_clean.json', 'mayo_clinic/mayo_clinic_data_clean.json')
